## Key Takeaways

### Core Principles

1. **Zero Config** 🚀
   - Works immediately with built-in components
   - No setup required to start learning

2. **Registries as Mappings** 📦
   - Component registries are just dicts
   - Easy to discover, access, and extend

3. **Mall Pattern** 🏬
   - Separate stores for each data type
   - Automatic persistence
   - MutableMapping interface everywhere

4. **Declarative Pipelines** ⚡
   - Compose components by name
   - DAG-based execution
   - Intermediate results collected automatically

5. **Progressive Enhancement** 🔧
   - Start with toys, add power as needed
   - Plugin system for production components
   - Optional dependencies

### Design Philosophy

`ef` follows these principles:
- **SSOT** (Single Source of Truth) - One place to store each piece of data
- **Open-Closed** - Easy to extend, hard to break
- **Dependency Injection** - Pass components in, don't hardcode
- **Facades** - Clean interfaces over complex functionality

### When to Use `ef`

✅ **Good for:**
- Learning about embedding pipelines
- Prototyping ML workflows
- Building reproducible experiments
- When you need automatic persistence
- When you want composable components

⚠️ **Consider alternatives when:**
- You need production-ready ML from the start (use imbed directly)
- You're building a full ML platform (use MLflow, Kubeflow)
- You need real-time streaming (ef is batch-oriented)

---

## Thanks for exploring `ef`! 🎉

Questions? Check out:
- 📖 [README.md](../README.md)
- 🧪 [tests/test_basic.py](../ef/tests/test_basic.py)
- 📝 [examples.py](../examples.py)
- 🔌 [Plugin Guide](../ef/plugins/README.md)

## 10. Working with File-Based Storage

To try things out you can use `backend='memory'`. 
But for real projects, use files:

In [ ]:
import tempfile
import os

from ef import Project

# Create a project with file-based storage
tmpdir = tempfile.mkdtemp()
print(f"{tmpdir=}")
project = Project.create('persistent_project', root_dir=tmpdir, backend='files')

# Add data and run pipeline
project.add_source('test_doc', 'This will be saved to disk')
_ = project.create_pipeline('simple', embedder='simple')
results = project.run_pipeline('simple')

# Check what got created
print("💾 File-based storage created:")
for root, dirs, files in os.walk(tmpdir):
    level = root.replace(tmpdir, '').count(os.sep)
    indent = ' ' * 2 * level
    print(f"{indent}{os.path.basename(root)}/")
    subindent = ' ' * 2 * (level + 1)
    for file in files[:5]:  # Show first 5 files
        print(f"{subindent}{file}")
    if len(files) > 5:
        print(f"{subindent}... and {len(files) - 5} more")

print("\n💡 With files backend:")
print("   - Data persists between sessions")
print("   - Different serialization for different data types")
print("   - segments → .txt files")
print("   - embeddings → .pkl files (pickle)")
print("   - planar_embeddings → .json files")

In [4]:
# Create different pipelines for different tasks
_ = project.create_pipeline('just_embed', embedder='simple')
_ = project.create_pipeline('embed_and_cluster', embedder='char_counts', clusterer='simple_kmeans', n_clusters=3)
_ = project.create_pipeline('full_viz', embedder='simple', planarizer='simple_2d', clusterer='threshold')

print("🔄 Multiple pipelines created:")
for i, pipeline_name in enumerate(project.list_pipelines(), 1):
    print(f"  {i}. {pipeline_name}")
    
print("\n💡 Each pipeline can use different components and parameters!")
print("   Run whichever one fits your current needs.")

🔄 Multiple pipelines created:
  1. simple
  2. just_embed
  3. embed_and_cluster
  4. full_viz

💡 Each pipeline can use different components and parameters!
   Run whichever one fits your current needs.


## 9. Advanced: Multiple Pipelines

You can create multiple pipelines for different purposes:

In [5]:
# Get comprehensive project summary
summary = project.summary()

print("📋 PROJECT SUMMARY")
print("=" * 60)
print(f"Project ID        : {summary['project_id']}")
print(f"Segments          : {summary['n_segments']}")
print(f"Embeddings        : {summary['n_embeddings']}")
print(f"Planar Embeddings : {summary['n_planar_embeddings']}")
print(f"Clusters          : {summary['n_clusters']}")
print(f"Pipelines         : {summary['n_pipelines']}")
print()
print("Available Components:")
for comp_type, names in summary['available_components'].items():
    print(f"  {comp_type:20} : {len(names)} ({', '.join(names[:3])}{', ...' if len(names) > 3 else ''})")

📋 PROJECT SUMMARY
Project ID        : persistent_project
Segments          : 1
Embeddings        : 1
Planar Embeddings : 0
Clusters          : 0
Pipelines         : 4

Available Components:
  segmenters           : 3 (identity, lines, sentences)
  embedders            : 2 (simple, char_counts)
  planarizers          : 2 (simple_2d, normalize_2d)
  clusterers           : 2 (simple_kmeans, threshold)


## 8. Project Summary - What Have We Built?

Let's see everything we've created:

### Component Registration

The `@register` decorator:
- Adds your function to the registry
- Stores metadata (like `dimension`)
- Makes it available for pipelines
- Validates the signature

You can register ANY callable - functions, lambdas, class methods, etc!

In [ ]:
# Use our custom embedder
_ = project.create_pipeline('word_analysis', embedder='word_stats')
custom_results = project.run_pipeline('word_analysis')

print("📊 Custom embedder results:")
print("   [words, avg_length, unique_words, sentences]")
for key, vector in list(custom_results['embeddings'].items())[:3]:
    print(f"  {key:12} : {vector}")

In [ ]:
# Register a custom embedder using a decorator
@project.embedders.register('word_stats', dimension=4)
def word_statistics_embedder(segments):
    """
    Create a 4D embedding: [word_count, avg_word_length, unique_words, sentence_count]
    """
    embeddings = {}
    for key, text in segments.items():
        words = text.split()
        embeddings[key] = [
            float(len(words)),                              # Total words
            sum(len(w) for w in words) / len(words) if words else 0,  # Avg word length
            float(len(set(words))),                         # Unique words
            float(text.count('.') + text.count('!') + text.count('?'))  # Sentences
        ]
    return embeddings

print("✅ Custom embedder 'word_stats' registered!")
print(f"   Available embedders now: {', '.join(project.embedders.keys())}")

## 7. Custom Components - Extend ef

You can easily add your own components! Let's create a custom embedder:

In [ ]:
# Quick one-off embedding (doesn't persist)
test_text = "This is a quick test of the embedding system"
embeddings = project.quick_embed(test_text, embedder='simple')

print("⚡ Quick embed (no pipeline needed!):")
print(f"  Input: '{test_text}'")
print(f"  Output: {embeddings['main']}")
print(f"\n💡 This didn't persist - check: 'main' in project.embeddings = {'main' in project.embeddings}")

## 6. Quick Embed - One-Off Operations

Sometimes you don't need a full pipeline - just a quick embedding. Use `quick_embed()`:

In [ ]:
# Look at cluster assignments
print("🎯 Cluster assignments:")
for key, cluster_id in project.clusters.items():
    doc_preview = project.segments[key][:40] + "..."
    print(f"  Cluster {cluster_id}: {key:12} → {doc_preview}")
    
print("\n💡 Documents in the same cluster are considered similar!")

In [ ]:
# Look at 2D coordinates (for visualization)
print("🗺️  Planar embeddings (2D coordinates):")
for key, coords in project.planar_embeddings.items():
    print(f"  {key}: ({coords[0]:.3f}, {coords[1]:.3f})")
    
print("\n💡 These 2D points could be plotted on a scatter plot!")

In [ ]:
# Look at embeddings (26D letter frequency vectors)
print("🔢 Embeddings (first 10 dimensions):")
for key, vector in list(project.embeddings.items())[:3]:
    print(f"  {key}: {vector[:10]}...")
    
print(f"\n📐 Each embedding has {len(list(project.embeddings.values())[0])} dimensions (a-z letter counts)")

## 5. Explore the Results

Let's examine what the pipeline produced:

### Pipeline Execution Explained

When you run a pipeline, ef:
1. **Loads** the data from `project.segments`
2. **Executes** each component in sequence (DAG-based execution)
3. **Collects** all intermediate results
4. **Persists** everything back to the mall automatically

The `results` dict contains ALL intermediate outputs - perfect for debugging and inspection!

In [ ]:
# Run the pipeline!
results = project.run_pipeline('full_analysis')

print("✅ Pipeline executed!")
print(f"\n📊 Results contain {len(results)} stages:")
for stage, data in results.items():
    print(f"  {stage:20} : {len(data)} items")
    
print("\n💾 All results automatically persisted to the mall!")

In [ ]:
# Create a full pipeline
_ = project.create_pipeline(
    'full_analysis',
    embedder='char_counts',      # Convert text to 26D vectors (letter frequencies)
    planarizer='normalize_2d',   # Reduce to 2D coordinates
    clusterer='simple_kmeans',   # Group into clusters
    n_clusters=2                 # Make 2 clusters
)

print("✅ Pipeline 'full_analysis' created!")
print("   Steps: segments → embeddings → planar_embeddings → clusters")
print(f"   Available pipelines: {', '.join(project.list_pipelines())}")

## 4. Create and Run a Pipeline

Now for the magic! ✨ Let's create a pipeline that:
1. Takes our documents (already segmented)
2. Embeds them into vectors
3. Reduces to 2D for visualization
4. Clusters similar documents together

### The Mall Pattern 🏬

Notice how we accessed `project.segments`? This is part of the **mall pattern**.

A "mall" is a **store of stores** - separate storage for each data type:
- `project.segments` - Original/segmented text
- `project.embeddings` - Vector representations
- `project.planar_embeddings` - 2D coordinates
- `project.clusters` - Cluster assignments

Each store is a **MutableMapping** (dict-like), so you can:
- Get/set items: `project.embeddings['doc1'] = [1.0, 2.0]`
- Iterate: `for key, value in project.embeddings.items(): ...`
- Check membership: `if 'doc1' in project.embeddings: ...`

In [ ]:
# Add multiple documents
documents = {
    'ai_intro': 'Artificial intelligence is transforming industries worldwide.',
    'ml_basics': 'Machine learning models learn patterns from data.',
    'dl_neural': 'Deep learning uses neural networks with many layers.',
    'nlp_text': 'Natural language processing helps computers understand human language.',
}

for key, text in documents.items():
    project.add_source(key, text)
    
print(f"✅ Added {len(documents)} documents")
print(f"📝 Total segments in store: {len(project.segments)}")

# Let's peek at what's stored
print("\n📂 Stored documents:")
for key in list(project.segments.keys())[:3]:
    preview = project.segments[key][:50] + "..." if len(project.segments[key]) > 50 else project.segments[key]
    print(f"  {key}: {preview}")

## 3. Add Source Data

Now let's add some actual data to process. We'll use a few documents about AI/ML topics:

In [ ]:
# Access a component directly (like a dict!)
simple_embedder = project.embedders['simple']

# Use it manually
test_segments = {
    'text1': 'Hello world',
    'text2': 'Machine learning is amazing!'
}

embeddings = simple_embedder(test_segments)

print("🔢 Embeddings from 'simple' embedder:")
for key, vector in embeddings.items():
    print(f"  {key}: {vector}")
    
print("\n💡 The 'simple' embedder counts: [chars, words, punctuation]")

### Understanding Component Types

- **Segmenters**: Split text into chunks (lines, sentences, etc.)
- **Embedders**: Convert text to vectors (numerical representations)
- **Planarizers**: Reduce high-dimensional vectors to 2D (for visualization)
- **Clusterers**: Group similar items together

Let's see how an embedder actually works:

In [ ]:
# ef (Embedding Flow) - Interactive Demo

Welcome to **ef**, a lightweight framework for building embedding pipelines!

## What is ef?

ef makes it easy to:
- 🔄 Transform data through embedding pipelines
- 📦 Store and retrieve results automatically
- 🔌 Compose components into workflows
- 🎯 Work immediately with built-in components
- 🔧 Extend with plugins when you need more power

## Why ef?

Traditional embedding workflows often require:
- Complex setup and configuration
- Manual data persistence
- Hardcoded pipeline steps
- Heavy dependencies upfront

**ef solves this** by providing:
- Zero-config startup (works immediately!)
- Automatic persistence via the "mall" pattern
- Declarative pipeline composition
- Progressive enhancement (start simple, add power as needed)

Let's dive in! 🚀

## Getting Started - Create Your First Project

A **Project** is your main workspace. It contains:
- 📋 **Component registries** - collections of functions for each pipeline stage
- 💾 **Data stores** - persistent storage for your data (the "mall")
- ⚡ **Pipelines** - composed workflows from your components

Let's create one!

In [ ]:
from ef import Project

# Create a project with in-memory storage (perfect for demos!)
project = Project.create('demo_project', backend='memory')

print("✅ Project created!")
print(f"Project ID: {project.project_id}")
print(f"Storage backend: memory (nothing written to disk)")

### What just happened?

When you create a project, ef automatically:
1. Sets up **component registries** (segmenters, embedders, planarizers, clusterers)
2. Registers **built-in toy components** so you can start immediately
3. Creates a **mall** (store of stores) for persisting data

The `backend='memory'` means everything stays in RAM - perfect for demos and testing!

For production, you'd use `backend='files'` to persist to disk.

## Discover Available Components

Component registries work like **dictionaries** - you can list, access, and register components.

Let's see what's available out of the box:

In [ ]:
# List all available components
components = project.list_components()

print("📦 Available Components:\n")
for component_type, names in components.items():
    print(f"  {component_type:20} : {', '.join(names)}")
    
print("\n💡 These are lightweight 'toy' implementations - perfect for learning!")
print("   (You can add production components via plugins later)")

## Next Steps - Production Use

### Built-in vs Production Components

The built-in components are **toy implementations** designed for learning and testing:
- `simple` embedder: Just counts characters, words, punctuation
- `char_counts` embedder: Letter frequency (26D)
- `simple_kmeans`: Basic clustering
- etc.

### Adding Production Components

For real applications, add the **imbed** plugin:

```python
from ef.plugins import imbed

# This would register production components:
# - OpenAI embeddings
# - Sentence transformers
# - Cohere embeddings
# - Advanced clustering algorithms
# - etc.

imbed.register(project)
```

### Install Full Dependencies

```bash
# Get everything
pip install ef[full]

# Get imbed integration
pip install ef[imbed]
```

### Other Extensions

- **dol**: Advanced storage backends (S3, Google Cloud, databases)
- **meshed**: Advanced DAG composition with caching
- **larder**: Automatic result caching and memoization